## Install Required Packages

In [1]:
!pip install Flask werkzeug  gpt4all huggingface-hub pypdf2 streamlit python-dotenv faiss-cpu chromadb langchain SpeechRecognition pydub langchain-together

## Imports

In [1]:
from langchain.llms import Together
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_together import Together
from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import os
import chromadb

## PDF Loader and Vector Database Creation

In [2]:
def load_chunk_persist_pdf() -> Chroma:
    pdf_folder_path = "./chat_pdf_data"
    documents = []
#     text = 'Who is Laurens van der Maaten?'
#     ST_embeddings = Embed4All('nomic-embed-text-v1.f16.gguf')
    ST_embeddings = gpt4all_embd = GPT4AllEmbeddings()
    

#     output = embedder.embed(text, prefix='search_document')
#     ST_embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())
    text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
    chunked_documents = text_splitter.split_documents(documents)
    client = chromadb.Client()
    if client.list_collections():
        consent_collection = client.create_collection("collection")
    else:
        print("Collection already exists")
    vectordb = Chroma.from_documents(
        documents=chunked_documents,
        embedding=ST_embeddings,
        persist_directory="./chroma_store"
    )
    vectordb.persist()
    return vectordb

## Agent Creation for Response Generation

In [3]:
def create_agent(context):

    # template = """
    # You are an expert accounting and a financial expert with expertise legislative knowledge of finance.
    # Explain the concept of {concept} in a couple of lines
    # """
    
    
    template = """
    As an expert with in-depth knowledge derived from key documents relevant to financial regulation and reporting in the UK, your role involves providing detailed and precise answers based on these texts. The documents you rely on include but are not limited to "Academies Accounts Direction 2022 to 2023," "Academies Model Accounts 2022 to 2023," "ATH 2023 Final," "Charities SORP FRS 102 (2019A)," "Companies Act 2006," "EFA Academies SORP Guide," "Framework and Guide for External Auditors 2022 to 2023," "FRS 100 Application of Financial Reporting Requirements (November 2022)," "FRS 103 Insurance Contracts," and "Redacted FRS 102 and FRS 105 (January 2022)."

Your answers must reflect a formal, professional tone, with each articulated in complete sentences ending with a full stop. Avoid harmful, disrespectful, or sensitive remarks.

When addressing inquiries:

1. Acknowledge the question or topic from the inquirer with a clear, initial sentence that concludes with a full stop. Clarify whether the query relates to regulatory practices, financial reporting standards, or other areas covered by the documents at your disposal.

2. Draw directly from the information within the specified documents to provide a detailed, informative response. If a question involves regulatory bodies such as the Financial Services Authority (FSA) and does not directly relate to the content of the listed documents, clearly state that the detailed answer is based on available information within the context of these documents and that specific regulatory inquiries may require consultation of direct regulatory advisories or legal guidelines.

3. Conclude your response with a summary of the key points or a concise recapitulation for clarity. Ensure the final sentence also ends with a full stop.

Context and previous interactions are as follows:
 
    Current Conversation
    Context: {context}
        
    {history}
    
    Human: {question}
    
    AI Assistant:
    
    """
    memory = ConversationBufferMemory(memory_key="history" ,input_key="question" )



    callback_manager = BaseCallbackManager([StreamingStdOutCallbackHandler()])
    # llm = GPT4All(model=gpt4all_path, max_tokens=1000,  callback_manager=callback_manager, verbose=True,repeat_last_n=0)

    llm = Together(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        temperature=0.5,
        callback_manager=callback_manager,
        max_tokens=600,
        top_k=1,
        top_p=1,        
        together_api_key="5ffb2e029349afeb3e62f25e83ea59499faa529df818c06782b7923e3a5e0aa1"
    )

    # callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


    # If you want to use a custom model add the backend parameter
    # Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
#     llm = GPT4All("./models/all-MiniLM-L6-v2-f16.gguf")

#     llm = LlamaCpp(
#         model_path="./models/mistral-7b-instruct-v0.1.Q4_0.gguf",
# #         n_ctx=5000,
# #         n_gpu_layers=1,
# #         n_batch=512,
#         f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
#         callback_manager=callback_manager,
#         verbose=True,

#     )

#     llm = GPT4All(model="./models/mistral-7b-instruct-v0.1.Q4_0.gguf",n_ctx=1000, callback_manager=callback_manager, verbose=False,repeat_last_n=0)
#     prompt = PromptTemplate(template=template, input_variables=["question","context","chat_history"]).partial(context=context)
    prompt = PromptTemplate(input_variables=["history", "question","context"], template=template).partial(context=context)
    llm_chain = LLMChain(prompt=prompt, llm=llm,verbose = False, memory=memory)

    return llm_chain

## Response Generation

In [4]:
def get_llm_response(query,context,vectordb):
    matching_docs = vectordb.similarity_search(query)
    context = ""

    for doc in matching_docs:
        context = context + doc.page_content + " \n\n "
    
    chain = create_agent(context)

    answer = chain.run(input_documents=matching_docs, question=query)
    return answer

## Initialize vector database (run only once as needed)

In [5]:
vectordb = load_chunk_persist_pdf()

Collection already exists


## Flask Backend

In [ ]:
from flask import Flask, render_template, request, jsonify
from werkzeug.serving import run_simple

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('indexx.html')

@app.route('/ask', methods=['POST'])
def ask():
    question = request.form['question']
    context = request.form['context']  # You might want to handle context from the front-end
    # Call your chatbot function here
    response = get_llm_response(question, context, vectordb)
    return jsonify({'response': response})

if __name__ == '__main__':
    run_simple('localhost', 5000, app)


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [19/May/2024 18:06:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 18:06:03] "GET /static/chatbot_logo.png HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 18:06:03] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/May/2024 18:06:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 18:06:09] "GET /static/chatbot_logo.png HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 18:06:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/May/2024 18:06:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 18:06:37] "GET /static/chatbot_logo.png HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 18:06:38] "GET /favicon.ico HTTP/1.1" 404 -
D:\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
127.0.0.1 - - [19/May/2024 18:06:46] "POST /ask HTTP/1.1" 200 -
127.0.0.1 - - [